In [1]:
import numpy as np
import pandas as pd
import geojson
import folium
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
from colour import Color
from PIL import Image

#### 1. Загрузка аггрегированных данных. Кол-во поездок из каждой зоны.

In [2]:
df = pd.read_csv('agg_data.csv')
regions = pd.read_csv('regions.csv', sep=';')

In [3]:
df.head()

tpep_pickup_datetime  region_1  region_2  region_3  region_4  region_5  \
0  2016-05-01 00:00:00       0.0       0.0       0.0       0.0       0.0   
1  2016-05-01 01:00:00       0.0       0.0       0.0       0.0       0.0   
2  2016-05-01 02:00:00       0.0       0.0       0.0       0.0       0.0   
3  2016-05-01 03:00:00       0.0       0.0       0.0       0.0       0.0   
4  2016-05-01 04:00:00       0.0       0.0       0.0       0.0       0.0   

   region_6  region_7  region_8  region_9  ...  region_2491  region_2492  \
0       0.0       0.0       0.0       0.0  ...          0.0          0.0   
1       0.0       0.0       0.0       0.0  ...          0.0          0.0   
2       0.0       0.0       0.0       0.0  ...          0.0          0.0   
3       0.0       0.0       0.0       0.0  ...          0.0          0.0   
4       0.0       0.0       0.0       0.0  ...          0.0          0.0   

   region_2493  region_2494  region_2495  region_2496  region_2497  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   region_2498  region_2499  region_2500  
0          0.0          0.0          0.0  
1          0.0          0.0          0.0  
2          0.0          0.0          0.0  
3          0.0          0.0          0.0  
4          0.0          0.0          0.0  

[5 rows x 2501 columns]

In [4]:
print('Общее кол-во поездок:', df.drop('tpep_pickup_datetime', axis=1).to_numpy().sum())

Общее кол-во поездок: 11626521.0


In [5]:
counter = 0
for column in df.drop('tpep_pickup_datetime', axis=1).columns:
    if df[column].sum() == 0:
        counter += 1
print('Количество ячеек с нулевым кол-вом поездок:', counter)

Количество ячеек с нулевым кол-вом поездок: 1283


#### 2. Статическая карта Нью-Йорка с Эмпайр-Стейт-Билдинг

In [6]:
def generateBaseMap(default_location=[40.748817, -73.985428], default_zoom_start=11):
    base_map = folium.Map(location=default_location, tiles='cartodbpositron', width=900, 
                          height=880, control_scale=True, zoom_start=default_zoom_start, 
                          no_touch=True,zoom_control=True)
    return base_map

In [7]:
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(geojson.Feature(geometry = geojson.Polygon([[(X['west'], X['south']),
        (X['east'], X['south']), (X['east'], X['north']), (X['west'], X['north'])]]), id = str(X.name), 
        properties=dict(name=X["region"], description=X["region"])))
    df.apply(insert_features, axis = 1)
    feature_collection = geojson.FeatureCollection(features)
    dumps = geojson.dumps(feature_collection)
    return geojson.loads(dumps)

In [8]:
geo_data = data2geojson(regions)

In [12]:
#Нарисуем интерактивную карту
base_map = generateBaseMap()
folium.Choropleth(geo_data=geo_data, fill_color='#00000000', line_opacity=0.2).add_to(base_map)
folium.Marker([40.748817, -73.985428]).add_to(base_map)
folium.map.Marker([40.748817, -73.985428], icon=folium.features.DivIcon(icon_size=(200,36), icon_anchor=(0,0),
        html='<div style="font-size: 9pt; color: Maroon"><b>%s</b></div>' % 'Empire State Building',)).add_to(base_map)
base_map

In [13]:
def save_map(geomap):
    geomap.save('map.html')
    delay=2
    fn='map.html'
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    base_map.save(fn)
    browser = webdriver.Chrome('/home/zea/chromedriver_linux64/chromedriver')
    browser.get(tmpurl)
    time.sleep(delay)
    browser.save_screenshot('map.png')
    im = Image.open('map.png')
    width, height = im.size 
    left = 54
    top = 116
    right = width-left-6
    bottom = height
    im = im.crop((left, top, right, bottom))
    im.save('map_in_png.png')
    browser.quit()

In [14]:
save_map(base_map)

In [15]:
def map_visualization(longitude, latitude,s=7):
    x_min=-74.25559
    x_max=-73.70001
    y_min=40.49612
    y_max=40.91553
    img = plt.imread('map_in_png.png')
    fig, ax = plt.subplots(figsize=(12, 12))
    fig.set_facecolor('.96')
    ax.imshow(img, extent = [x_min, x_max, y_min,y_max])
    ax.scatter(x=longitude, y=latitude, c='g' , s=s, label='Taxi pickup point')
    ax.legend(loc='upper left', fontsize=16)
    plt.title('New York city map', size=20, y=1.03)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    ax.xaxis.grid(True, which='major', linewidth=0.1,
    color='black', linestyle='--')
    ax.yaxis.grid(True, which='major', linewidth=0.1,
    color='black', linestyle='--')

In [16]:
#Нарисуем статическую карту по условию задания
map_visualization(-73.985428, 40.748817)

#### 3. Визуализация поездок из каждой ячейки

In [17]:
grouped = df.drop('tpep_pickup_datetime', axis=1).sum().to_frame()
grouped = grouped.reset_index()
grouped = grouped.rename(columns={'index': 'region', 0: 'count'})
grouped['count'] = grouped['count'].astype('int')
grouped['region'] = grouped['region'].apply(lambda x: x.split('region_')[1])
grouped['region'] = grouped['region'].astype('str')

In [20]:
base_map = generateBaseMap()
choropleth=folium.Choropleth(geo_data=geo_data, data=grouped, columns=['region', 'count'],
    key_on='feature.id', fill_color='YlOrBr', nan_fill_color='#00000000', fill_opacity=0.6,
    line_opacity=0.2, smooth_factor=3, legend_name='Taxi trips count', reset=True).add_to(base_map)
style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False))
folium.Marker([40.748817, -73.985428]).add_to(base_map)
folium.map.Marker([40.748817, -73.985428], icon=folium.features.DivIcon(icon_size=(200,36), icon_anchor=(0,0),
        html='<div style="font-size: 9pt; color: Maroon"><b>%s</b></div>' % 'Empire State Building',)).add_to(base_map)
folium.LayerControl().add_to(base_map)

In [21]:
#Интерактивная карта
base_map

In [22]:
#Статическая карта
save_map(base_map)

trips_data = np.array(grouped['count'])
coord_data = np.array(regions)

color_1 = 'LightYellow'
color_2 = 'Brown'
color_1 = Color(color_1)
grad = list(color_1.range_to(Color(color_2), int(trips_data.max())))

fig, ax = plt.subplots(figsize=(12, 1.5))
fig.set_facecolor('.96')
trips_values = list(range(0, int(trips_data.max())+1, 500))
colorbar = np.array([[tuple([int(round(j*255, 0)) for j in grad[int(i)-1].rgb]) for i in trips_values] for _ in range(100)])
ax.imshow(colorbar)
plt.yticks([])
ticks = list(range(len(trips_values)))
plt.xticks(ticks=ticks[::100] + [ticks[-1]], labels=trips_values[::100] + [trips_values[-1]])
plt.title('Taxi trip counts heatmap', y=1.01, fontsize=20)
plt.show()
map_visualization(-73.985428, 40.748817, 70)

#### 4. Интерактивная карта

In [23]:
base_map

#### 5. Интерактивная карта со средним значением поездок в ячейке за месяц

In [24]:
grouped_mean = df.drop('tpep_pickup_datetime', axis=1).mean().to_frame()
grouped_mean = grouped_mean.reset_index()
grouped_mean = grouped_mean.rename(columns={'index': 'region', 0: 'mean'})
grouped_mean['region'] = grouped_mean['region'].apply(lambda x: x.split('region_')[1])
grouped_mean['region'] = grouped_mean['region'].astype('str')

In [27]:
base_map = generateBaseMap()
choropleth=folium.Choropleth(geo_data=geo_data, data=grouped_mean, columns=['region', 'mean'],
    key_on='feature.id', fill_color='YlOrBr', nan_fill_color='#00000000', fill_opacity=0.6,
    line_opacity=0.2, smooth_factor=3, legend_name='Taxi trips count', reset=True).add_to(base_map)
style_function = "font-size: 15px; font-weight: bold"
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name'], style=style_function, labels=False))
folium.Marker([40.748817, -73.985428]).add_to(base_map)
folium.map.Marker([40.748817, -73.985428], icon=folium.features.DivIcon(icon_size=(200,36), icon_anchor=(0,0),
        html='<div style="font-size: 9pt; color: Maroon"><b>%s</b></div>' % 'Empire State Building',)).add_to(base_map)
folium.LayerControl().add_to(base_map)

In [28]:
base_map

#### 6. Отфильтруем ячейки

In [29]:
grouped_mean_filtered = grouped_mean[grouped_mean['mean'] >= 5]

In [30]:
grouped_mean_filtered.head()

region        mean
1073   1074   66.799731
1074   1075  138.266129
1075   1076   86.802419
1123   1124   66.041667
1124   1125  181.892473

In [31]:
print('Кол-во ячеек со средним больше 5:', grouped_mean_filtered.shape[0])

Кол-во ячеек со средним больше 5: 102
